# <center>OpenAI agent pattern: basic agent</center>

A starter guide for building a basic agent with tool calling using the `openai-agents` library. 

Here we've setup a basic agent that can answer questions about stocks using `web_search`. 

### Install Libraries

In [3]:
# Install base libraries for OpenAI
!pip install -q openai openai-agents

# Install optional libraries for OpenInference/OpenTelemetry tracing
!pip install -q arize-phoenix-otel openinference-instrumentation-openai-agents openinference-instrumentation-openai

### Setup Keys

Add your OpenAI API key to the environment variable `OPENAI_API_KEY`.

Copy your Phoenix `API_KEY` from your settings page at [app.phoenix.arize.com](https://app.phoenix.arize.com).

In [4]:
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass(
    "🔑 Enter your OpenAI API key: "
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Setup Tracing

In [5]:
from phoenix.otel import register

tracer_provider = register(
    project_name="openai-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: openai-agents
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Create your basic agent

In [20]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Finance Agent",
    instructions="You are a finance agent that can answer questions about stocks. Use web search to retrieve up‑to‑date context. Then, return a brief, concise answer that is one sentence long.",
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
)

In [22]:
from pprint import pprint
result = await Runner.run(agent, "what is the latest news on Apple?")

# Get the final output
print(result.final_output)

# Get the entire list of messages recorded to generate the final output
pprint(result.to_input_list())

Apple is exploring integrating AI-powered search engines into Safari, potentially challenging Google's search market dominance. ([reuters.com](https://www.reuters.com/business/apple-looks-add-ai-search-companys-browser-bloomberg-reports-2025-05-07/?utm_source=openai))

## Stock market information for Apple Inc (AAPL)
- Apple Inc is a equity in the USA market.
- The price is 197.52 USD currently with a change of -0.99 USD (-0.00%) from the previous close.
- The latest open price was 199.05 USD and the intraday volume is 11859568.
- The intraday high is 200.98 USD and the intraday low is 197.52 USD.
- The latest trade time is Wednesday, May 7, 14:56:07 UTC.
 
[{'content': 'what is the latest news on Apple?', 'role': 'user'},
 {'id': 'ws_681b98d0a9d88192b498440c896bf6e90c2e237e8d4cf4bd',
  'status': 'completed',
  'type': 'web_search_call'},
 {'content': [{'annotations': [{'end_index': 268,
                                'start_index': 128,
                                'title': 'Apple